# APIs and SQL

*This notebook inclues adapted content from [Melanie Walsh's chapter on Data Collection](https://melaniewalsh.github.io/Intro-Cultural-Analytics/04-Data-Collection/00-Data-Collection.html).*

In this lab, we'll introduce a useful way to extract data from online, as well as a canonical tool used to explore large datasets when you don't have access to a Python environment. We'll go over the following topics:

- Accessing an API
- API Wrappers
- SQL and SQLite
- pandasql

# APIs

It seems only natural that we should be able to extract any data from the internet by programmatically logging information after "going" to each website you're interested in. (In fact, it is [perfectly legal](https://melaniewalsh.github.io/Intro-Cultural-Analytics/04-Data-Collection/01-User-Ethics-Legal-Concerns.html).) One way to do this is using [web scraping](https://melaniewalsh.github.io/Intro-Cultural-Analytics/04-Data-Collection/02-Web-Scraping-Part1.html), where you write an algorithm which parses website content, logs data, and loops through several HTML web pages. But, this method is becoming less effective over the years, as websites are becoming far more complex (harder to scrape), and most companies are transitioning to a platform where their data is more easily accessible (and controlled) in an Application Programming Interface (API).

## What is an API?

**An API allows you to programmatically extract and interact with company data which drives their websites.** In this way, social networks, museums, foundations, research labs, applications, and projects can make their data publicly available, allowing for developers to use the data to build applications and tools (e.g., for your phone, computer, or refrigerator) that can be used by the general populous. For example, the reason you can access Google Maps on your phone is because developers used the Google Maps API to build that functionality.

Of course, there are plenty of companies or foundations which will likely never use APIs to store/access their data. In these cases though, you can usually find an API that is *related* to that website, or someone may have built (or, they are building) a third-party API for that purpose. Web scraping should typically be a last resort, so we do not teach it in this class.

<span style="color:red">**Caveat:** People typically design their APIs such that they decide exactly which kinds of data they want to share. So, they often choose not to share their most lucrative and desirable data. In those cases, you are usually asked to pay some fee.</span>

## Using Environment Variables

We will discuss environment variables more in a future lesson, but to use APIs properly, we need to have at least a basic understanding of what environment variables are.

When working on any data science project (e.g., like the web app you'll build later in this course), you will likely track your progress using Git/GitHub. But, keys and secret strings (like the ones we will use to access an API) should never be pushed to GitHub. Instead, it's a best practice to use environment variables when dealing with this kind of data. In short, environment variables are values stored in a special file on your local computer, or on the cloud where your project may be hosted. In this way, those variables are only accessible to agents with access to that file (e.g., your Python interpreter, or the one on the cloud).

In this class, we will use [dotenv](https://github.com/theskumar/python-dotenv#getting-started) to manage environment variables for API. You'll need to `pip` install it, as directed in the instructions, then create a file with the name *.env* (notice the period) in your project directory to hold any keys or secrets. Since we're going to use this package in this notebook, we'll import the library here. *Note: If you're using Git/GitHub, make sure ".env" is added to your [.gitignore file](https://www.atlassian.com/git/tutorials/saving-changes/gitignore)*.

In [3]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv

## Accessing an API

The steps to access *any* API are about the same, no matter the API. So, in this lesson, we're going to use the [Genius](https://genius.com/) API to access data about songs.

### Step 1: Client Access Token

Typically, to use an API, you need a special API key usually called a "Client Access Token", which is kind of like a password. Many APIs require authentication keys to gain access to them. To get your necessary Genius API keys, follow these steps:

1. Navigate to the [api-clients page](https://genius.com/api-clients) (which will prompt you to [sign up for an account](https://genius.com/signup_or_login) if you haven't already). Then, click the button that says **"New API Client"**.
2. Remember, APIs are expecting *developers* to use their APIs to build applications (e.g., for your phone, computers, etc.). But, since we're only doing data analysis for a college course in informatics, we only need to fill in the fields for "App Name" (e.g., *"Song Lyrics Project"*), and "App Website URL" (e.g., *"https://github.com/leontoddjohnson/i501"*). Then, click **Save**.
3. When you click "Save," you'll be given a series of API Keys: a "Client ID" and a "Client Secret." **Copy/Paste these values into your *.env* file** without quotations, as instructed in the dotenv documentation. For example, my *.env* file looks something like this:
    CLIENT_ID=asdfghjkl;123456789
    CLIENT_SECRET=qwertyuiop098765432
6. To generate your "Client Access Token," which is the API key that we'll be using in this notebook, you need to click "Generate Access Token". Place that in your *.env* file as you did the other variables, maybe under the variable name ACCESS_TOKEN.

We can access our `ACCESS_TOKEN` by using *dotenv* to load our environment variables into the current environment, then with the built-in Python *os* library to access it.

In [5]:
load_dotenv()

False

In [ ]:
import os
# do not print this variable anywhere if the notebook is going on GitHub
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']

### Step 2: Making an API Request

Making an API request is very similar to accessing a URL in your browser. But, instead of getting a rendered HTML web page in return, you get some data in return.

There are a few different ways that we can [query the Genius API](https://docs.genius.com/#songs-h2), but here we'll use [the basic search](https://docs.genius.com/#search-h2), which allows you to get a bunch of Genius data about any artist or songs that you search for:

`http://api.genius.com/search?q={search_term}&access_token={client_access_token}`

First we're going to assign the string "Missy Elliott" to the variable `search_term`. Then we're going to make an f-string URL that contains the variables we'd like to include in our query.

In [ ]:
search_term = "Missy Elliott"

In [ ]:
genius_search_url = f"http://api.genius.com/search?q={search_term}&access_token={ACCESS_TOKEN}"

You can see the data we'll be requesting from this API by printing the `genius_search_url`, pasting it into your browser.

In [ ]:
# print(genius_search_url)

The data you might see when you navigate to your URL is in [JSON](https://www.w3schools.com/whatis/whatis_json.asp) format. JSON is an acronym for JavaScript Object Notation, and it is a data format commonly used by APIs. JSON data can be nested, and contains key-value pairs, much like a Python dictionary.

We can access this JSON directly in Python using the [`requests` library](https://requests.readthedocs.io/en/latest/) to send HTTP requests to a remote client. If you like, you can read more about what a "request" is [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview), but it suffices to say that it represents an online communication between your computer and the server storing the data you want.

In [ ]:
import requests

In [ ]:
# here, we make a "GET" request to the Genius server
response = requests.get(genius_search_url)
json_data = response.json()

In [ ]:
json_data

In [ ]:
json_data['response'].keys()

Genius places all of its search results into the "hits" element. By default, it looks like it returns at most 10 search results for any request.

In [ ]:
len(json_data['response']['hits'])

According to the documentation, we can use [referents](https://docs.genius.com/#referents-h2) to increase that number to a maximum of 20 results per request using `per_page`. With this slight adjustment added, let's consolidate our request into a single function to use again later. We'll also add this to our *api_util.py* file for the lab.

In [ ]:
def genius(search_term, per_page=15):
    '''
    Collect data from the Genius API by searching for `search_term`.
    
    **Assumes ACCESS_TOKEN is loaded in environment.**
    '''
    genius_search_url = f"http://api.genius.com/search?q={search_term}&" + \
                        f"access_token={ACCESS_TOKEN}&per_page={per_page}"
    
    response = requests.get(genius_search_url)
    json_data = response.json()
    
    return json_data['response']['hits']

In [ ]:
json_data = genius("The Beatles")
len(json_data)

## Loading JSON Data Into a DataFrame

For us to efficiently work with the JSON data, we need to load them into a DataFrame. Using panda's [read_json function](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html), we can do just that in a pretty efficient way. The only detail is we need the JSON to be in one of the acceptable orientations (see the `orient` argument in the documentation).

In [ ]:
import pandas as pd
import json

In [ ]:
json_data[0]

When we look at any of the hits, we see the data we're interested in is contained in the `"result"` element. We can consolidate all of the "result" elements for each "hit" using a list comprehension. We then use the `json` library to convert this list of JSONs into a single JSON.

**Looking ahead:** Notice that the `"stats"` and the `"primary_artist"` elements contain *dictionaries* of interesting data that we'll need to unpack once we have our data into a DataFrame.

In [ ]:
hits = [hit['result'] for hit in json_data]
hits_json = json.dumps(hits)

# load JSON into DataFrame
df = pd.read_json(hits_json)

In [ ]:
df.head(2)

Recall that `"stats"` and `"primary_artist"` contain dictionaries which we want to unpack. After a bit of StackOverflow searching (say), we find that we can [use](https://stackoverflow.com/a/38231651) `pd.apply(pd.Series)` and `pd.concat` to explode these into columns. We'll need to make a slight adjustment to the column names to avoid repeats.

In [ ]:
df_stats = df['stats'].apply(pd.Series)
df_stats.rename(columns={c:'stat_' + c for c in df_stats.columns},
                inplace=True)

df_primary = df['primary_artist'].apply(pd.Series)
df_primary.rename(columns={c:'primary_artist_' + c for c in df_primary.columns},
                  inplace=True)

df = pd.concat((df, df_stats, df_primary), axis=1)

In [ ]:
df[['stat_unreviewed_annotations', 'stat_hot', 'stat_pageviews']]

### Collecting Multiple API Calls

We are going to want to perform analysis on more than one artist, so let's use what we've written above to collect data from multiple API calls by looping through multiple search terms. When we loop through each search term, we use the [tqdm package](https://pypi.org/project/tqdm/) to help us visualize our progress (you may need to use `pip` to install it). This kind of thing is helpful when we're running multiple API calls, and we don't know how long it will take.

In [ ]:
from tqdm import tqdm

In [ ]:
search_terms = ['The Beatles', 'Missy Elliot', 'Andy Shauf', 'Slowdive', 'Men I Trust']
n_results_per_term = 10

dfs = []

# loop through search_terms in question
for search_term in tqdm(search_terms):
    json_data = genius(search_term, per_page=n_results_per_term)
    hits = [hit['result'] for hit in json_data]
    hits_json = json.dumps(hits)

    # load JSON into DataFrame
    df = pd.read_json(hits_json)

    # expand dictionary elements
    df_stats = df['stats'].apply(pd.Series)
    df_stats.rename(columns={c:'stat_' + c for c in df_stats.columns},
                    inplace=True)
    
    df_primary = df['primary_artist'].apply(pd.Series)
    df_primary.rename(columns={c:'primary_artist_' + c for c in df_primary.columns},
                      inplace=True)
    
    df = pd.concat((df, df_stats, df_primary), axis=1)
    
    # add to list of DataFrames
    dfs.append(df)

In [ ]:
df_genius = pd.concat(dfs)

In [ ]:
df_genius.shape

In [ ]:
df_genius.sample(3)

Of course, we'll want to copy this function into (or build it *within* our IDE in) our .py file.  

## Using an API Wrapper

More often than not, someone has built an "API Wrapper" for the API you are working with. An API wrapper makes an API easier to use, and it often extends the API itself. It will typically consist of classes and functions similar to the ones we've built above, but spanning a wide range of functionality and access to the API. For example, John Miller's [LyricsGenius](https://github.com/johnwmillr/LyricsGenius) gives us an almost universal access to the Genius website, and it even uses web scraping to collect song lyrics themselves.

<span style="color: darkblue">**If ever you're working with an API, do some Googling to make sure there isn't a wrapper you can use to make things easier on you!**</span>

First, we'll [install LyricsGenius](https://github.com/johnwmillr/LyricsGenius#installation) *(in 2023, there is no conda option, so we would need to use `pip`)*, then import it.

In [ ]:
import lyricsgenius

In [ ]:
# creating an "API Class" is typical for API wrappers
LyricsGenius = lyricsgenius.Genius(ACCESS_TOKEN)

To get the top songs and song lyrics from a specific artist you can use the method `.search_artist()`:

In [ ]:
artist = LyricsGenius.search_artist("Missy Elliott", max_songs=2)

In [ ]:
print(artist.songs[0].lyrics[:300])

You'll notice this function took *much* longer than our function above. If you take a quick glance at the documentation for the function (use Shift+Tab in the parentheses next to the function), you'll see that the `get_full_info=True` argument slows down the search (likely because it includes scraping lyrics). If we were using the lyrics in our investigation, we might be okay with this delay, but since we're only interested in numerical data for the time being (and because setting `get_full_info=False` is still a bit slow), we'll continue using the process we built above.

# SQL

*<span style="color:darkred;">Section Prerequisites: [SQLBolt](https://sqlbolt.com/) lessons 1-6 (and, 7-12 if possible).</span>*

[Structured Query Language (SQL)](https://www.sqltutorial.org/what-is-sql/) is a programming language designed to allow users to **query** databases containing multiple tables (rows and columns) of data, each related to one another using column-to-column relationships. It is easily the most popular language for accessing large tabular databases, so naturally, many companies and users have used the language to create their own variants of the language called [dialects](https://arctype.com/blog/sql-dialects/). In this class, we will be using [SQLite](https://www.sqlite.org/index.html) and its corresponding dialect.

In the job setting, you will typically make SQL queries using a combination of the following:

- Some data browser, with data stored on the cloud or on in-house servers
- Python, likely using a package called SQLAlchemy to connect to a database

In this class, we will use [DB Browser](https://sqlitebrowser.org/about/) to simulate the kind of browser you'd use on the job, and SQLAlchemy to gain practice with the tool. We'll also see how you can use the [pandasql package](https://pypi.org/project/pandasql/) to assimilate SQL queries into the pandas framework.

## Set Up

### SQLite

Before we can continue, you need to make sure that SQLite is installed on your machine.

- For MacOS users, SQLite should already be installed on your computer. You can test this by running `sqlite3 --version` in your terminal.
- For everyone else, you'll need to [follow these steps to download and install SQLite](https://www.sqlitetutorial.net/download-install-sqlite/).

Lastly, for this lab, we're going to use the [SQLite Sample Database](https://www.sqlitetutorial.net/download-install-sqlite/). Scroll down to the "Download SQLite sample database" section of the page for the link, or download it directly [here (as of Sep 2023)](https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip). Unzip the file, and move the .db file to a convenient location (e.g., the same place where this lab is saved). *Note: If you are using GitHub, ".db" should typically be added to your [.gitignore file](https://www.atlassian.com/git/tutorials/saving-changes/gitignore).*

### SQLAlchemy and pandasql

Lastly, we'll be using [SQLAlchemy](https://www.sqlalchemy.org/) to connect our Python environment to our database, and [pandasql](https://pypi.org/project/pandasql/) to use SQL "in" pandas. You can install them both using anaconda:

    pip install SQLAlchemy 
    pip install -U pandasql

### DB Browser 

Next, you'll need to [install DB Browser](https://sqlitebrowser.org/dl/) by following the installation instructions provided for your particular operating system. *Note: For Mac "M1/M2" chips, you'll use the "Apple Silicon" option.*

Once installed, double click on the DB Browser for SQLite icon to open up the application on your computer. On MacOS, you may get an error message saying that the application can't be opened because Apple cannot check it for malicious software. To resolve, right click on the DB Browser for SQLite icon and click Open instead.

**Exploring in DB Browser**

Let's explore the `chinook` database (i.e., the SQLite Sample Database) using DB Browser to "test" queries, and SQLAlchemy (below) to run queries here in the notebook.

1. First, in DB Browser, click the "Open Database" button, then find the *chinook.db* file on your computer. (This is how you would open any SQL ".db" file.)
2. Close the side panels on the right until all you see is the Main window and the handy "DB Schema" viewer on the right. The [SQL Schema](https://www.sqlite.org/schematab.html) provides information on the tables and columns within a database. *Note: you can also view a SQL database schema directly using [PRAGMA commands](https://www.sqlite.org/pragma.html).*
3. Select the "Execute SQL" tab to start writing SQL Queries.

## SQLAlchemy

Again, you can use DB Browser to explore your data, but you can also *bring that data into your notebook* using a combination of SQLAlchemy and pandas.

### Connect

To connect to a database using SQLAlchemy, we need to define the database location. Using `create_engine`, we create a connection between the SQL database represented in the *.db* file, and our Python instance.

*Note: In our case, we have a database immediately accessible on our computer. But in practice, you'll more likely need to [access a remote database](https://docs.sqlalchemy.org/en/20/core/engines.html#custom-dbapi-connect-arguments-on-connect-routines) requiring credentials.*

In [ ]:
import os  # if you haven't already, above

from sqlalchemy import inspect, create_engine
import pandas as pd

For this lab, the database is stored in a *data* folder inside the same directory as this notebook. This notebook has a "working directory" or file path associated with it, which can be used by Python to "navigate" to the same location. Using the same `os` library from above, we can use `os.getcwd()` to get the **current working directory**, and use it to navigate to the *chinook* database.

In [ ]:
cwd = os.getcwd()
db_path = cwd + "/data/chinook.db"  # complete path to the database file

In [ ]:
# the "engine" is a connection between Python and the database
engine = create_engine(f"sqlite:////{db_path}")

In [ ]:
# this is one way to access an aspect of the schema
insp = inspect(engine)

In [ ]:
insp.get_table_names()

### Load into pandas

Once you have a connection between Python and the database, we can use `pd.read_sql()` to load the result of SQL queries into pandas.

In [ ]:
query = \
'''
SELECT
    DISTINCT(city)
FROM employees;
'''

df_result = pd.read_sql(query, engine)
df_result

## SQL Statements

<span style="color: darkred;">*This section contains code examples from the [SQLite Tutorial Website](https://www.sqlitetutorial.net/). Refer to this website for more in-depth explanations.*</span>

We interact with SQL using a "query", or the code/interface between the user and the database. It contains keywords, column names, tables names, and even function operations. In this notebook, we will introduce a couple of examples of some common query statements, and then follow up with a few more methods SQL provides.

*Note: **SQL code is case-insensitive**, but I find it to be a good practice to capitalize keywords (e.g., "SELECT"), and lower the case of column names (e.g., "employees", above) when using SQL. I also tend to use different lines and indenting wherever possible to keep the code clean.*

### [SELECT](https://www.sqlitetutorial.net/sqlite-select/)

The foundation of virtually all SQL queries is the `SELECT` statement. Typically, this is followed (at some point) by a `FROM`, denoting (naturally) where we are selecting our data from. `DISTINCT` removes duplicate rows of a column.

In [ ]:
query = \
'''
SELECT
    DISTINCT city
FROM employees;
'''

df_result = pd.read_sql(query, engine)
df_result

### [LIMIT](https://www.sqlitetutorial.net/sqlite-limit/)

`LIMIT` only returns the first set of rows for a result, very much like `head()`.

In [ ]:
query = \
'''
SELECT
	name,
	composer,
	milliseconds,
	unitprice
FROM tracks
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
    name,
    composer,
    unitprice
FROM tracks
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

### [ORDER BY](https://www.sqlitetutorial.net/sqlite-order-by/)

We can also order our data based on some column (akin to "sort", in pandas). Sort will default to ascending order, but it's a good practice to include `ASC` or `DESC` as needed.

In [ ]:
query = \
'''
SELECT
	name,
	milliseconds, 
	albumid
FROM
	tracks
ORDER BY
	albumid ASC,
    milliseconds DESC;
'''

df_result = pd.read_sql(query, engine)
df_result

*Note: the column you use to sort your data *does not* need to be included in the SELECT statement.*

In [ ]:
query = \
'''
SELECT
	name,
	composer,
	albumid
FROM
	tracks
ORDER BY
    milliseconds DESC
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

Using `LIMIT` in tandem with `ORDER BY` helps us extract the `n`th item (highest or lowest), ordered by some column.

In [ ]:
# second longest track
query = \
'''
SELECT
	trackid,
	name,
	milliseconds
FROM
	tracks
ORDER BY
	milliseconds DESC
LIMIT 1 OFFSET 2;
'''

df_result = pd.read_sql(query, engine)
df_result

### [WHERE](https://www.sqlitetutorial.net/sqlite-where/)

We can *filter* our data using the `WHERE` clause. In the same way that pandas provides logical operations, there are also several available in SQLite (see the section link above for more on these, and [this article on "glob" operators](https://www.sqlitetutorial.net/sqlite-glob/)).

In [ ]:
query = \
'''
SELECT
	name,
	albumid,
	Milliseconds,
	mediatypeid
FROM
	tracks
WHERE
	mediatypeid IN (2, 3)
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

The `%` wildcard can be handy.

In [ ]:
query = \
'''
SELECT
	name,
	albumid,
	composer
FROM
	tracks
WHERE
	composer LIKE '%Smith%'
ORDER BY
	albumid;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	name,
	milliseconds,
	bytes,
	albumid
FROM
	tracks
WHERE
	albumid = 1
	AND milliseconds > 250000;
'''

df_result = pd.read_sql(query, engine)
df_result

### [IS (NOT) NULL](https://www.sqlitetutorial.net/sqlite-is-null/)

Of course, we may want to include or exclude missing values in the data. In SQL, "missing" values are encoded as `NULL`.

In [ ]:
query = \
'''
SELECT
    Name, 
    Composer
FROM
    tracks
WHERE
    Composer IS NOT NULL
ORDER BY 
    Name;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	InvoiceId,
	BillingCity,
	BillingState,
	BillingPostalCode,
	Total
FROM invoices
WHERE
	BillingState IS NULL
	AND BillingPostalCode IS NULL
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

### [JOIN](https://www.sqlitetutorial.net/sqlite-join/)

The SQL JOIN allows us to merge data from multiple tables. It's essentially the same thing as the `pandas.merge`, but the code is a bit more accessible than pandas when it comes to merging many tables together.

One common practice is for databases to have an "entity" table, which contains the ID along with many other attributes of the entity. Then, in a table, one only needs to reference the ID of the entity rather than store redundant data that exists already in another table.

#### [INNER JOIN](https://www.sqlitetutorial.net/sqlite-inner-join/)

The `INNER JOIN` only matches rows where the column value in question exists in **both** tables.

In [ ]:
query = \
'''
SELECT 
    ar.Name artist_name,
    al.Title AS album_title
FROM 
    albums al
INNER JOIN artists ar
    ON al.ArtistId = ar.ArtistId
ORDER BY ar.Name
LIMIT 15;
'''

df_result = pd.read_sql(query, engine)
df_result

A few things to note here:

- the `AS` keyword (or a space) followed by some string or keyword allows us to change the way data is presented in the result of a query, such as for renaming columns or tables.
- Whenever we join multiple tables, it's a good practice to "name" those tables (using the `AS`/space syntax), then reference columns with the `table.column` notation.

#### [LEFT JOIN](https://www.sqlitetutorial.net/sqlite-left-join/)

Similarly, the `LEFT JOIN` collects all rows where the value in question exists in the "left" table, regardless of whether it exists in the "right" table. (Left and right here are defined by the order in which tables occur on either side of the word `JOIN`).

In [ ]:
query = \
'''
SELECT
    ar.Name artist_name,
    al.Title album_title
FROM
    artists ar
LEFT JOIN albums al ON
    ar.ArtistId = al.ArtistId
WHERE al.Title IS NULL   
ORDER BY Name
LIMIT 5;
'''

df_result = pd.read_sql(query, engine)
df_result

#### [CROSS JOIN](https://www.sqlitetutorial.net/sqlite-cross-join/)

The `CROSS JOIN` collects all combinations of values between two columns in a table. This kind of function is handy when you want to calculate something for multiple groups based on all the values that exist.

In [ ]:
query = \
'''
SELECT * 
FROM media_types
CROSS JOIN genres
LIMIT 50;
'''

df_result = pd.read_sql(query, engine)
df_result

#### [FULL OUTER JOIN](https://www.sqlitetutorial.net/sqlite-full-outer-join/)

The `FULL OUTER JOIN` collects the *union* of all rows which have matching columns values between tables.

In [ ]:
query = \
'''
SELECT
    ar.Name artist_name,
    al.Title album_title
FROM
    artists ar
FULL OUTER JOIN albums al ON
    ar.ArtistId = al.ArtistId
ORDER BY Name
'''

df_result = pd.read_sql(query, engine)
print("Number of rows: ", df_result.shape[0])
df_result

*Note: there are only 275 rows in the `artists` table, and 347 in the `albums` table. We'll see how you can calculate these values shortly!*

#### [SELF JOIN](https://www.sqlitetutorial.net/sqlite-self-join/)

The `SELF JOIN` is just a join between a table and itself.

In [ ]:
query = \
'''
SELECT m.firstname || ' ' || m.lastname AS 'manager',
       e.firstname || ' ' || e.lastname AS 'direct_report' 
FROM employees e
INNER JOIN employees m
    ON m.employeeid = e.reportsto
ORDER BY manager;
'''

df_result = pd.read_sql(query, engine)
df_result

### [GROUP BY](https://www.sqlitetutorial.net/sqlite-group-by/)

The `GROUP BY` function exists across many data manipulation frameworks (e..g, R, pandas, etc.), and it is meant to break up the data into groups. Typically, once data is broken into groups, continuous values are aggregated to a single value within each group. SQL provides many [aggregation functions](https://www.sqlitetutorial.net/sqlite-aggregate-functions/) which can be used with `GROUP BY`.

In [ ]:
query = \
'''
SELECT
	albumid,
	COUNT(trackid)
FROM
	tracks
GROUP BY
	albumid
ORDER BY COUNT(trackid) DESC;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	t.albumid AS album_ID,
	a.title AS album_name,
	COUNT(t.trackid) AS num_track_ids
FROM
	tracks t
INNER JOIN albums a
    ON a.albumid = t.albumid
GROUP BY
	t.albumid
ORDER BY
	num_track_ids DESC
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

### [HAVING](https://www.sqlitetutorial.net/sqlite-having/)

The `HAVING` operator is the `WHERE` operator which we can apply *after* the `GROUP BY`. That is, the "Group By Section" of a query has keywords in this order: `WHERE` $\to$ `GROUP BY` $\to$ `HAVING`.

A good way to remember this is that the word "having" makes more sense if you think about it as applied to *collections* (or groups) of things rather than the things themselves (e.g., "I *have* a handful of marbles", not "I *where* a handful of marbles").

In [ ]:
query = \
'''
SELECT
   albumid,
   COUNT(trackid)
FROM
   tracks
GROUP BY
   albumid
HAVING 
   COUNT(albumid) BETWEEN 18 AND 20
ORDER BY albumid;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	ar.name AS artist_name,
	a.title AS album_name,
	COUNT(trackid) AS num_tracks
FROM
	tracks t
INNER JOIN albums a
	ON t.albumid = a.albumid
LEFT JOIN artists ar
	ON a.ArtistId = ar.ArtistId
WHERE
	artist_name LIKE "%Jam%"
GROUP BY
	a.ArtistId,
	t.albumid
HAVING
	num_tracks > 10
ORDER BY
	artist_name ASC,
	num_tracks DESC
'''

df_result = pd.read_sql(query, engine)
df_result

### [CASE](https://www.sqlitetutorial.net/sqlite-case/)

The SQL `CASE` statement is the analog for if-then-else operations in Python.

In [ ]:
query = \
'''
SELECT customerid,
	firstname,
	lastname,
    country,
	CASE country 
		WHEN 'USA' 
			THEN 'Domestic' 
			ELSE 'Foreign' 
	END CustomerGroup
FROM 
    customers
ORDER BY 
    LastName,
    FirstName
LIMIT 20;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	trackid,
	name,
	CASE
		WHEN milliseconds < 60000
			THEN 'short'
		WHEN milliseconds > 60000 
		AND milliseconds < 300000
			THEN 'medium'
		ELSE
			'long'
		END category
FROM
	tracks
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

## Subqueries and Views

Sometimes, it's helpful to *use* the result of one query *within* another query. This is typically called a [**Subquery**](https://www.sqlitetutorial.net/sqlite-subquery/).

- The [(NOT) EXISTS](https://www.sqlitetutorial.net/sqlite-exists/) operator checks whether a subquery returns a result at all.
- If a subquery is overly complex, or if you plan to use it in the future, you can save it  as a [**view**](https://www.sqlitetutorial.net/sqlite-create-view/) (or, you can [delete](https://www.sqlitetutorial.net/sqlite-drop-view/) one you no longer need).

As an example, below we have a subquery which returns the track information for only a particular album.

In [ ]:
query = \
'''
SELECT trackid,
       name,
       albumid
FROM tracks
WHERE albumid = (
   SELECT albumid
   FROM albums
   WHERE title = 'Let There Be Rock'
);
'''

df_result = pd.read_sql(query, engine)
df_result

### Set Operations

In SQL, there are also set operations [UNION](https://www.sqlitetutorial.net/sqlite-union/), [EXCEPT](https://www.sqlitetutorial.net/sqlite-except/) (i.e., set difference), and [INTERSECT](https://www.sqlitetutorial.net/sqlite-intersect/). For each of these, you'd use subqueries to build the query.

### EXERCISE

Take a look at these different set operations. Can you build a query which returns a **single column** of the unique album names *and* artist names which contain the word "black"?

In [ ]:
# your code here

## SQL Functions

It's rare that we are satisfied with the data as it exists within the data table. Typically, we want to transform the data, and present it in a certain way. This is where SQL Functions come in.

### Mathematical Operations

SQLite has several different [data types](https://www.sqlitetutorial.net/sqlite-data-types/), and sometimes, we'd like to leverage one type over another. Suppose we'd rather show the number of minutes rather than milliseconds. We can use the `CAST` operator to convert our value to a `FLOAT`, or we can divide *by* a float (e.g., `60000.0`) to coerce our data into the more complex `FLOAT` data type.

In [ ]:
query = \
'''
SELECT
	name,
	albumid,
	CAST(Milliseconds AS FLOAT) / 60000 minutes,
	mediatypeid
FROM
	tracks
WHERE
	mediatypeid IN (2, 3)
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	name,
	albumid,
	ROUND(Milliseconds / 60000.0, 3) AS minutes,
	mediatypeid
FROM
	tracks
WHERE
	mediatypeid IN (2, 3)
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

### [Date Functions](https://www.sqlitetutorial.net/sqlite-date-functions/)

Dates come with their own "numerical" representation which can be operated on. In SQL, we can calculate different date-based values using datetime functions.

In [ ]:
query = \
'''
SELECT
	LastName,
	FirstName,
	title,
	BirthDate,
	strftime('%m', BirthDate) BirthMonth,
	HireDate
FROM employees
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	LastName,
	FirstName,
	title,
	HireDate,
    DATE(HireDate,
		'start of month', 
		'+1 month', 
		'-1 day') last_day_of_hire_month
FROM employees;
'''

df_result = pd.read_sql(query, engine)
df_result

### [String Functions](https://www.sqlitetutorial.net/sqlite-string-functions/)

Strings are very versitile, and SQL has plenty of operations for handling them. For example, we can use `LENGTH` to determine the lengths of the names for some of these playlists.

In [ ]:
query = \
'''
SELECT
	Name name,
	LENGTH(name) name_length
FROM playlists
ORDER BY name_length DESC
LIMIT 10;
'''

df_result = pd.read_sql(query, engine)
df_result

In [ ]:
query = \
'''
SELECT
	FirstName,
	LastName,
	REPLACE(
		REPLACE(Title, "Manager", "Boss"),
		"IT", "Computer") slang_title
FROM employees;
'''

df_result = pd.read_sql(query, engine)
df_result

### [Window Functions](https://www.sqlitetutorial.net/sqlite-window-functions/)

Window functions perform calculations on rows of data **based on their row-index**. For instance, we might want to know how a row of data compares to others with values lower than it, or maybe a just the index of the row itself, or even a cumulative sum. The syntax for the query looks a bit like this:

```sqlite
SELECT
    ...,
    [SOME_EXPRESSION](columns_of_stuff)   --<-- Here lies the "window function"
        OVER (                            --<-- Apply this function *OVER* some window
            PARTION BY ... ) AS ...       --<-- Define the "window" and the final column name
FROM ...
```

We perform operations on the records that are inside the window. The `PARTITION` tells you what is included in the window.

For instance, we can use it to perform a similar task as `.transform`. This query tells us how far a `Total` Invoice amount is from the average total for its city.

In [ ]:
query = \
'''
SELECT 
    CustomerId,
    InvoiceDate,
    BillingCity,
    Total - AVG(Total) OVER (
        PARTITION BY BillingCity
      ) AS diff_from_city_avg
FROM invoices
ORDER BY CustomerId, InvoiceDate;
'''

df_result = pd.read_sql(query, engine)
df_result

Or, we can use it to calculate a **cumulative** sum.

In [ ]:
query = \
'''
SELECT 
	CustomerId,
    InvoiceDate,
    BillingCity,
	TOTAL,
    SUM(Total) OVER (
        PARTITION BY CustomerId 
        ORDER BY InvoiceDate
    ) AS customer_running_total
FROM invoices
ORDER BY CustomerId, InvoiceDate
LIMIT 20;
'''

df_result = pd.read_sql(query, engine)
df_result

And (among other things), we could assign a rank to each track of each album based on the length of that track in comparison to the others in the album.

In [ ]:
query = \
'''
SELECT
	Name,
	Milliseconds,
	AlbumId,
	RANK () OVER ( 
		PARTITION BY AlbumId
		ORDER BY Milliseconds DESC
	) LengthRank 
FROM tracks
LIMIT 50;
'''

df_result = pd.read_sql(query, engine)
df_result

# EXERCISES

## Problem 1

In fact, it looks like we can use the `page` referent to capture up to 20 results for *multiple pages* of results (think of scrolling through search results), and append each page to a collection of final results. Is this possible? If so, adjust the above function to include the `page` referent in Genius to return more than 20 results for a search term. If not, explain why.

In [ ]:
# your code here

## Problem 2

Write a SQL query which provides the minimum, maximum, and average track count of albums for each genre. So, each row should be a genre, and the columns would reflect the minimum track count, maximum track count, and average track count. *Feel free to use DB Brower as your "scratchpad" to test out your code.*

In [ ]:
# your code here

## Problem 3

Using window functions and the `chinook` database, write a query which tells us the time between each invoice for each customer in the `invoices` table. E.g., you might have a column that says "time_since_last_invoice".

In [ ]:
# your code here

## Problem 4

Take a look at the documentation for [pandasql](https://pypi.org/project/pandasql/). Load in any data frame of your choosing, and select a column that best represents a unique identifier for each row. E.g., if my data frame contains a list of customers, I might use the customer name or customer ID. Then, use *pandasql* to run a SQL query which performs a self join on your data based on that unique identifier column.

In [ ]:
# your code here